In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


To install dependencies

Note: After installation, please restart the runtime if it is required.

In [ ]:
!pip install ultralytics
!pip install av
!pip install filterpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=c22be3b6ad3e36b14fbc62a346d532fc3a8f30f97d5f07cce527db87f529675c
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
Successfully built filterpy


The Player & ball tracking with norfair

In [ ]:
import sys
root_path = '/content/drive/MyDrive/basketball-broadcaster'
sys.path.append(root_path)

import os
os.chdir(root_path)

import av
import time
from PIL import Image
import numpy as np

from pb_detector import PBDetector
from pb_tracker import PBTracker

input_ = av.open("videos/test1_clip.mp4")
output = av.open("output.mp4", "w")

detector = PBDetector()
tracker = PBTracker()

# Make an output video stream using the input video stream as a template
in_video_stream = input_.streams.video[0]
out_video_stream = output.add_stream(codec_name="h264",
                                      width=in_video_stream.width,
                                      height=in_video_stream.height,
                                      rate=int(in_video_stream.average_rate),
                                      bit_rate=in_video_stream.bit_rate,
                                      pix_fmt=in_video_stream.pix_fmt)

# Make an output audio stream using the input audio stream as a template
in_audio_stream = input_.streams.audio[0]
out_audio_stream = output.add_stream(template=in_audio_stream)

frame_idx = 0
det_cycle = 3

st = time.time()
for packet in input_.demux():
    # We need to skip the "flushing" packets that `demux` generates.
    if packet.dts is None:
        continue

    if packet.stream == in_video_stream:
        for frame in packet.decode():
            # To get an input frame
            image = frame.to_image() # PIL image
            image = np.array(image)
            ##############  To process by AI engine  ############

            # To process the input frame
            player_detections = []
            ball_detections = []
            if frame_idx % det_cycle == 0:
                player_detections, ball_detections = detector.detect(image)

            player_detections, ball_detections = tracker.run(image, player_detections, ball_detections)
            frame_idx += 1

            # To draw the result
            image = Image.fromarray(image)
            image = tracker.draw_detections(image, player_detections, ball_detections)

            ####################################################

            output_frame = av.VideoFrame.from_image(image)
            output_frame.pts = None
            for output_packet in out_video_stream.encode(output_frame):
                # Write the packets to the container
                output_packet.stream = out_video_stream
                output.mux(output_packet)
    elif packet.stream == in_audio_stream:
        packet.stream = out_audio_stream
        output.mux(packet)

elapsed = time.time() - st

input_.close()
output.close()

In [ ]:
# To print the FPS
print('Processing FPS: ', int(in_video_stream.frames / elapsed))

Processing FPS:  3
